In [1]:
import tensorflow as tf
import scipy.io as scio
import scipy.sparse as sp
import pandas as pd
import numpy as np

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
import tensorflow as tf

In [3]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [4]:
import tensorflow as tf
from tensorflow.keras import activations, regularizers, constraints, initializers

class HGT_Layer_dynamic(tf.keras.layers.Layer):
    def __init__(self,
                 m,
                 activation=lambda x: x,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 **kwargs):
        super(HGT_Layer_dynamic, self).__init__()

        self.m = m #要降成的维度
        self.activation = activations.get(activation)
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        
        
        self.convolutions = []
  


    def build(self, input_shape):
        #print(input_shape)

        """
        HGT_Layer has three inputs : [shape(AnCount,An), shape(X_count,X),shape(Z_count,Z)]
        X is self_feature,Z is another node type feature
        
        """
        Adj_set=input_shape[0]  #输入一个list，元素为一个邻接矩阵
        X_shape  =input_shape[1]
        Z_shape  =input_shape[2]
        
        #设置Q,K,V Q权重矩阵,因为输入的是多个邻接矩阵，所以对应多个Q,K,V,使用list分别存储
        
        
        

        self.weight_Q=self.add_weight(name="weight_Q",
                        shape=(X_shape[1], self.m),
                        initializer=self.kernel_initializer,
                        trainable=True)
        
            
        
        
        self.weight_K_T=self.add_weight(name="weight_K_T",
                        shape=(self.m,Z_shape[1]),
                        initializer=self.kernel_initializer,
                        trainable=True)
            
       
        
        
        
        
        self.weight_V=self.add_weight(name="weight_V",
                        shape=(Z_shape[1], self.m),
                        initializer=self.kernel_initializer,
                        trainable=True)
            
            
        
        
           
        self.weight_ATT=self.add_weight(name="weight_V",
                        shape=(self.m, self.m),
                        initializer=self.kernel_initializer,
                        trainable=True)
            

    def call(self, inputs):
        """ HGT_Layer has three inputs : [An, X，Z]
        """
        
        self.X = inputs[1]
        self.Z = inputs[2]
        self.Z_T=tf.transpose(self.Z)
        self.An=inputs[0] 
        
        if self.An.shape[0]!=self.X.shape[0]:     
            self.An=tf.sparse.transpose(self.An)


            
        ##################################################################
        self.Q=tf.matmul(self.X,self.weight_Q)
        self.K_T=tf.matmul(self.weight_K_T,self.Z_T)
        self.V=tf.matmul(self.Z,self.weight_V)
        
        ##################################################################
        Att=tf.matmul(self.Q,self.weight_ATT)
        Att=tf.matmul(Att,self.K_T)
        Att=Att*self.An  #An是一个邻接矩阵，做一个哈达玛积,是一个sparse.Tensor
        Att=tf.sparse.softmax(Att)
        ##################################################################

        output=tf.sparse.sparse_dense_matmul(Att,self.V)
            
        if self.activation:
            output=self.activation(output)
        ##################################################################

        return output

In [5]:
class Self_Attention(tf.keras.layers.Layer):
    def __init__(self,
                 att_size,
                 activation=lambda x: x,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',):
        super(Self_Attention, self).__init__()
        self.activation = activations.get(activation)
        self.att_size=att_size
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        
    def build(self, input_shape):
        
        weight_att_shape_x=input_shape[-1]
        weight_att_shape_y=self.att_size
        
        self.weight_att=self.add_weight(name="weight_to_att",
                               shape=(weight_att_shape_x, weight_att_shape_y),
                               initializer=self.kernel_initializer, 
                               trainable=True)
        
        self.bias=self.add_weight(name="bias",
                               shape=(weight_att_shape_y,),
                               initializer=self.bias_initializer,
                               trainable=True)
        
        self.weight_att_u=self.add_weight(name="weight_to_att_u",
                               shape=(weight_att_shape_y, 1),
                               initializer=self.kernel_initializer,
                               trainable=True)

    
    def call(self, inputs):

        self.X = inputs
        
        WX=tf.matmul(self.X,self.weight_att)
        V=tf.tanh(tf.add(WX,self.bias))
        VU=tf.matmul(V,self.weight_att_u)
        
        alphas=tf.nn.softmax(VU,axis=0) 

        
        outputs=tf.multiply(alphas,self.X)
    
        
        outputs_att=tf.reduce_sum(outputs,axis=0) 
        
        if self.activation:
            outputs_att=self.activation(outputs_att)
        
#         print('inputs.shape=',inputs.shape)
#         print('VU.shape=',VU.shape)
#         print('alphas.shape=',alphas.shape)
        alphas_new=tf.transpose(tf.squeeze(alphas, axis=-1)) #需要改变attention形状

        return outputs_att,alphas_new
    

In [6]:
# 稀疏矩阵转稀疏张量
def sp_matrix_to_sp_tensor(M):
    if not isinstance(M, sp.csr.csr_matrix):
        M = M.tocsr()
    # 获取非0元素坐标
    row, col = M.nonzero()
    # SparseTensor参数：二维坐标数组，数据，形状
    X = tf.SparseTensor(np.mat([row, col]).T, M.data, M.shape)
    X = tf.cast(X, tf.float32)
    return X

In [7]:
class Multi_HGT_Model(tf.keras.Model):
    def __init__(self,
                 Muilt_AA,
                 Muilt_AP,
                 Muilt_AT,
                 Muilt_AC,
             
                 
                 Muilt_FA,
                 Muilt_FP,
                 Muilt_FT,
                 Muilt_FC,
    
                 output_size=3): 

        # 调用父类__init__()方法
        super(Multi_HGT_Model, self).__init__()
        
        self.Muilt_AA=Muilt_AA
        self.Muilt_AP=Muilt_AP
        self.Muilt_AT=Muilt_AT
        self.Muilt_AC=Muilt_AC
        
        
        
        self.Muilt_FA =Muilt_FA
        self.Muilt_FP =Muilt_FP
        self.Muilt_FT =Muilt_FT
        self.Muilt_FC =Muilt_FC
     
        

        
        #同质图
        self.HGT_layer_list=[]
        for i in range(len(self.Muilt_AA)):
            self.HGT_layer_list.append(
                HGT_Layer_dynamic(4,activation=tf.keras.activations.gelu)
            )
            
            
        #二部图
        self.HGT_layer_list_bio_AP=[]
        for i in range(len(self.Muilt_AP)):
            self.HGT_layer_list_bio_AP.append(
                HGT_Layer_dynamic(4,activation=tf.keras.activations.gelu)
            )
            
        self.HGT_layer_list_bio_AT=[]
        for i in range(len(self.Muilt_AT)):
            self.HGT_layer_list_bio_AT.append(
                HGT_Layer_dynamic(4,activation=tf.keras.activations.gelu)
            )
        
        self.HGT_layer_list_bio_AC=[]
        for i in range(len(self.Muilt_AC)):
            self.HGT_layer_list_bio_AC.append(
                HGT_Layer_dynamic(4,activation=tf.keras.activations.gelu)
            )
            

        

  
        
        #注意力
        self.att_layer = Self_Attention(4,activation=tf.keras.activations.gelu)


        #Decoder部分
        self.dense=tf.keras.layers.Dense(output_size,activation='softmax')

        




    def call(self,input_x_id, training=False,dropout=0.):
        # 输入数据
        
        #同质网络I 编码过程
        
        _h1_for_A_list=[]
        for i in range(len(self.Muilt_AA)):
            _h1_for_A_list.append(
                self.HGT_layer_list[i]([self.Muilt_AA[i], self.Muilt_FA,self.Muilt_FA]) 
            
            )
            



         #二部图AP 编码过程
        _h2_for_A_list=[]
        for i in range(len(self.Muilt_AP)):
            _h2_for_A_list.append(
                self.HGT_layer_list_bio_AP[i]([self.Muilt_AP[i], self.Muilt_FA,self.Muilt_FP])   
            
            )
            
        #二部图AT 编码过程
        _h3_for_A_list=[]
        for i in range(len(self.Muilt_AT)):
            _h3_for_A_list.append(
                self.HGT_layer_list_bio_AT[i]([self.Muilt_AT[i], self.Muilt_FA,self.Muilt_FT])   
            
            )
        
        #二部图AC 编码过程
        _h4_for_A_list=[]
        for i in range(len(self.Muilt_AC)):
            _h4_for_A_list.append(
                self.HGT_layer_list_bio_AC[i]([self.Muilt_AC[i], self.Muilt_FA,self.Muilt_FC])   
            
            )

        
        
       
        #各个子图结果汇总
        _h_for_A_list_set=tf.stack(_h1_for_A_list+_h2_for_A_list+_h3_for_A_list+_h4_for_A_list,axis=0)


   
        _h7_for_A,att_for_A=self.att_layer(_h_for_A_list_set) 

        
        

        
        #解码层
        output=self.dense(_h7_for_A) 
        
     
        
        
        ###########################################################################
        output_list=[]
        for idx in input_x_id:
            output_list.append(output[idx]) #返回指定id的所有向量
        ########################################################################     

       
        #att=[att_for_A,att_for_A_2,att_for_A_3,att_for_A_4]
        att=[att_for_A]
        ######################################################################### 
 
        
       
            
        output=[output_list,]
        embedding=_h7_for_A
        
        return output,att,embedding

# 加入数据测试一下

In [8]:
#获取文件夹下所有文件名
import os
path=r'C:\Users\Hone\Desktop\Experimental_code\DBLP_2\计算的各个元路径矩阵'
name_list=[]
filelist=os.listdir(path)
for filename in filelist:
    name_list.append(filename)

In [9]:
#取出矩阵的名字
matrix_name_list=[]
for name in name_list:
    matrix_name_list.append(name.split('.')[0].split('_matrix_')[1])

In [10]:
matrix_name_list

['APA',
 'APCP',
 'APCPA',
 'APCPT',
 'APTP',
 'APTPA',
 'APTPC',
 'CPA',
 'CPAPA',
 'PA',
 'PAPA',
 'TPA',
 'TPAPA',
 'TPTPA']

In [11]:
def find_matrix_from_file(path,key_name):
    filelist=os.listdir(path)
    
    for name in filelist:
        if key_name==name.split('.')[0].split('_matrix_')[1]:
            Matrix = sp.load_npz(path+'\\'+name) #读取
            return Matrix
        else:
            pass
        

    print('cant find %s in %s' %key_name %path)
    

In [12]:
#需要将矩阵按照类型进行区分，放进不同的dict
data_AA={}
data_AP={}
data_AT={}
data_AC={}



for pathname in matrix_name_list:
    if pathname[0]==pathname[-1]: #同质邻接矩阵
        data_AA[pathname]=find_matrix_from_file(path,pathname) #输入目标位置文件，与指定文件名，返回该稀疏矩阵
    elif pathname[0]=='P' or pathname[-1]=='P':
        data_AP[pathname]=find_matrix_from_file(path,pathname) 
    elif pathname[0]=='T' or pathname[-1]=='T':
        data_AT[pathname]=find_matrix_from_file(path,pathname) 
    elif pathname[0]=='C' or pathname[-1]=='C':
        data_AC[pathname]=find_matrix_from_file(path,pathname)


In [13]:
Paths_name_AA=[keyword for keyword in data_AA]
Paths_name_AP=[keyword for keyword in data_AP]
Paths_name_AT=[keyword for keyword in data_AT]
Paths_name_AC=[keyword for keyword in data_AC]

In [14]:
AA_matrix=[]
for name in Paths_name_AA:
    AA_matrix.append(data_AA[name])

In [15]:
def adj_list(name,data_dict):
    matepath_adj_list=[]
    for i in name:
        matepath_adj_list.append(sp_matrix_to_sp_tensor(data_dict[i])) #转tensor.sparse
    return matepath_adj_list

In [16]:
AA_matrix=adj_list(Paths_name_AA,data_AA)
AP_matrix=adj_list(Paths_name_AP,data_AP)
AT_matrix=adj_list(Paths_name_AT,data_AT)
AC_matrix=adj_list(Paths_name_AC,data_AC)

In [17]:
import numpy as np

In [18]:
FA=np.random.rand(4057,128)
FP=np.random.rand(14376,128)
FT=np.random.rand(8920,128)
FC=np.random.rand(20,128)

In [19]:
# FA=np.random.rand(4057,128)
# FP=np.eye(14376)
# FT=np.eye(8920)
# FC=np.eye(20)

In [20]:
model=Multi_HGT_Model(AA_matrix,
                           AP_matrix,
                           AT_matrix,
                           AC_matrix,
                      
                           
                           FA,
                           FP,
                           FT,
                           FC,
                                                  
                           output_size=4
                          )


In [21]:
#输入训练id集合
import random

index_list_for_FA=[i for i in range(4057)] #需要随机化
random.shuffle(index_list_for_FA) #index乱序
    
a_part=int(len(index_list_for_FA)/10)
train_idx_for_FA = index_list_for_FA[:int(a_part*2)]  # 2 1 7
val_idx_for_FA = index_list_for_FA[ int(a_part*2)+1:int(a_part*3)]
test_idx_for_FA = index_list_for_FA[ int(a_part*3)+1:len(index_list_for_FA)]

In [22]:
output,att,embedding=model(index_list_for_FA,training=True,dropout=0.)

In [23]:
model.summary()

Model: "multi_hgt__model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hgt__layer_dynamic (HGT_Lay  multiple                 1552      
 er_dynamic)                                                     
                                                                 
 hgt__layer_dynamic_1 (HGT_L  multiple                 1552      
 ayer_dynamic)                                                   
                                                                 
 hgt__layer_dynamic_2 (HGT_L  multiple                 1552      
 ayer_dynamic)                                                   
                                                                 
 hgt__layer_dynamic_3 (HGT_L  multiple                 1552      
 ayer_dynamic)                                                   
                                                                 
 hgt__layer_dynamic_4 (HGT_L  multiple            

In [24]:
#标签
data_label=sp.load_npz(r'C:\Users\Hone\Desktop\Experimental_code\DBLP_2\sparse_matrix_author_label.npz') #读取
data_label=data_label.todense()
data_label=np.array(data_label)

In [25]:
optimizer=tf.keras.optimizers.Adam() #learning_rate=0.01
loss_func=tf.keras.losses.CategoricalCrossentropy()
#这个loss函数还不够全，还得加入L2正则化


train_loss=tf.keras.metrics.Mean('train_loss') #计算loss的均值
train_acc_I=tf.keras.metrics.CategoricalAccuracy('train_accuracy_I')


test_loss=tf.keras.metrics.Mean('test_loss')
test_acc_I=tf.keras.metrics.CategoricalAccuracy('test_accuracy_I')

In [26]:
def train_step(model,id_x,labels_x,dropout=0.):  #一个set为一个batch的量，
    #tf.GradientTape()记录运算过程的变量梯度
    
    with tf.GradientTape() as t:
        output_list,att,embedding=model(input_x_id=id_x,training=True,dropout=dropout)    #计算一个batch的预测值   #!!!
     
        loss_step_1=loss_func(labels_x,output_list[0])   #计算一个batch的损失函数
        loss_step=loss_step_1
        
 
    grads=t.gradient(loss_step,model.trainable_variables)  #更新变量的梯度
    
    
    #利用跟新的梯度计算变量，model.trainable_variables表示网络中所有的变量
 
    optimizer.apply_gradients(zip(grads,model.trainable_variables))  
    train_loss(loss_step)  #传入计算一个epoch累计loss函数中
    train_acc_I(labels_x,output_list) #传入计算一个epoch累计accuracy函数中
    
    
    return att

In [27]:
def test_step(model,id_x,labels_x): 
    output_list,att,embedding=model(input_x_id=id_x,training=False,dropout=0.) 
    
    loss_step_1=loss_func(labels_x,output_list[0])   #计算一个batch的损失函数
    #loss_step_2=loss_func(labels_x,output_list[1])   
    loss_step=loss_step_1
    
    test_loss(loss_step)
    test_acc_I(labels_x,output_list) #传入计算一个epoch累计accuracy函数中

In [28]:
def split_list_by_n(list_collection, n):
    """
    将集合均分，每份n个元素
    :param list_collection:
    :param n:
    :return:返回的结果为评分后的每份可迭代对象
    """
    for i in range(0, len(list_collection), n):
        yield list_collection[i: i + n]

In [29]:
def train(
          AA_matrix,
          AP_matrix,
          AT_matrix,
          AC_matrix,
       
    
          FA,
          FP,
          FT,
          FC,
     
    
          epoch_time,  
          batch_size,
          
          
          id_x_set, 
          id_x_set_for_val,
          id_x_set_for_test,
          
  
          label_set_I,
          output_size=3,
          dropout=0.
         ): #需要给要训练的集合和验证集合
    
    model=Multi_HGT_Model(AA_matrix,
                               AP_matrix,
                               AT_matrix,
                               AC_matrix,
                         
                               
                               FA,
                               FP,
                               FT,
                               FC,
         
                               
                               output_size=output_size
                               )
    

    best_acc_result=0
    for epoch in range(epoch_time):
        
        #training
        batch_idx_train=[] #获取一个batch的数据
        for i in split_list_by_n(id_x_set,batch_size): #split_list_by_n这个函数
            batch_idx_train.append(i)  
                     
        #创建batch_label矩阵

        
        for a_batch_x in batch_idx_train: 
            
            batch_label_list_x=[]

            for id_x in a_batch_x:
                batch_label_list_x.append(label_set_I[id_x]) 
                
    

            att=train_step(model,
                           id_x=a_batch_x,
                           labels_x=batch_label_list_x,
                           dropout=dropout) 
            print('一个train_batch训练完毕')
                           
        print('Epoch {} ，train_loss={:.3f}，train_acc_I={:.3f}'.format(epoch,
                                                                        train_loss.result(),
                                                                        train_acc_I.result())) 
        
        
        
        
        
        #validating
        batch_idx_val=[] #获取一个batch的数据
        
        
        
        for i in split_list_by_n(id_x_set_for_val,batch_size):
            batch_idx_val.append(i)
            

            
        for a_batch_x in batch_idx_val:
            batch_label_list_x=[]
            
            for id_x in a_batch_x:
                batch_label_list_x.append(label_set_I[id_x]) #获取一个batch的所有label

                
            
            test_step(model,id_x=a_batch_x, labels_x=batch_label_list_x)
            print('一个val_batch验证完毕')
            
        print('val_loss={:.3f}，val_accuracy_I={:.3f},best_accuracy={:.3f}'.format(test_loss.result(),test_acc_I.result(),best_acc_result)
             )
        
        ####################################################################################
        #保存结果最佳的embedding
        test_acc=float(test_acc_I.result())
        if test_acc>best_acc_result:
            
            best_acc_result=test_acc
           
            
            
            if best_acc_result>0.6:
                
                #需要返回，train数据载入得到的att
                att=train_step(model,
                    id_x=id_x_set,
                    labels_x=label_set_I[id_x_set]) 
    
                #输出test数据集的embedding 
                #输入test数据集，输出test数据集的embedding
                _,att_test,embedding=model(input_x_id=id_x_set_for_test,
                          training=False) 
                #还需要一个对应的label矩阵
                #从label_set_I中取出对应id的值，重新组成一个新的矩阵
                embedding_list=[]
                new_label_matrix_list=[]
                for i in id_x_set_for_test:
                    new_label_matrix_list.append(label_set_I[i])
                    embedding_list.append(embedding[i])
                    new_label_matrix=np.array(new_label_matrix_list)
    
                #保存embedding和对应的label矩阵
                np.save(r'C:\Users\Hone\Desktop\My_model-DATA\DBLP\embedding_of_test_data_two_loss_HGT_20220503',np.array(embedding_list))
                np.save(r'C:\Users\Hone\Desktop\My_model-DATA\DBLP\label_matrix_of_test_data_two_loss_HGT_20220503',new_label_matrix)
                print('表示向量保存完毕！')

        train_loss.reset_states()
        train_acc_I.reset_states()

        
        test_loss.reset_states()
        test_acc_I.reset_states()
        ####################################################################################
        
        
        
    #循环结束,对test数据验证
    batch_idx_test = []  # 获取一个batch的数据

    
    for i in split_list_by_n(id_x_set_for_test, batch_size):
        batch_idx_test.append(i)

    

    for a_batch_x in batch_idx_test:
        batch_label_list_x = []

        
        for id_x in a_batch_x:
            batch_label_list_x.append(label_set_I[id_x])  # 获取一个batch的所有label
    
        

        test_step(model,id_x=a_batch_x,labels_x=batch_label_list_x)
    print('End of model training\n'
          'test_loss={:.3f}，test_accuracy_I={:.3f}'.format(test_loss.result(),
                                                          test_acc_I.result()))
    

    test_loss.reset_states()
    test_acc_I.reset_states()

          
    
    
    
    

          
    return att

In [30]:
att=train(
    AA_matrix=AA_matrix,
    AP_matrix=AP_matrix,
    AT_matrix=AT_matrix,
    AC_matrix=AC_matrix,
 
    
    FA=FA,
    FP=FP,
    FT=FT,
    FC=FC,

   
    epoch_time=50,  
    batch_size=1000,
          
          
    id_x_set          =train_idx_for_FA, 
    id_x_set_for_val  =val_idx_for_FA,
    id_x_set_for_test =test_idx_for_FA,

  
    label_set_I=data_label,
    output_size=data_label.shape[1],
    dropout=0.
)

一个train_batch训练完毕
Epoch 0 ，train_loss=1.381，train_acc_I=0.298
一个val_batch验证完毕
val_loss=1.386，val_accuracy_I=0.295,best_accuracy=0.000
一个train_batch训练完毕
Epoch 1 ，train_loss=1.377，train_acc_I=0.305
一个val_batch验证完毕
val_loss=1.385，val_accuracy_I=0.297,best_accuracy=0.295
一个train_batch训练完毕
Epoch 2 ，train_loss=1.374，train_acc_I=0.302
一个val_batch验证完毕
val_loss=1.384，val_accuracy_I=0.297,best_accuracy=0.297
一个train_batch训练完毕
Epoch 3 ，train_loss=1.371，train_acc_I=0.319
一个val_batch验证完毕
val_loss=1.384，val_accuracy_I=0.317,best_accuracy=0.297
一个train_batch训练完毕
Epoch 4 ，train_loss=1.368，train_acc_I=0.325
一个val_batch验证完毕
val_loss=1.384，val_accuracy_I=0.314,best_accuracy=0.317
一个train_batch训练完毕
Epoch 5 ，train_loss=1.366，train_acc_I=0.338
一个val_batch验证完毕
val_loss=1.385，val_accuracy_I=0.317,best_accuracy=0.317
一个train_batch训练完毕
Epoch 6 ，train_loss=1.364，train_acc_I=0.356
一个val_batch验证完毕
val_loss=1.386，val_accuracy_I=0.317,best_accuracy=0.317
一个train_batch训练完毕
Epoch 7 ，train_loss=1.362，train_acc_I=0.370


In [31]:
att_1=tf.reduce_sum(att[0],axis=0)/att[0].shape[0]
att_weihgt_1=list(np.array(att_1))
for i in att_weihgt_1:
    print('%.3f' %i)

0.063
0.054
0.032
0.071
0.032
0.050
0.065
0.043
0.039
0.119
0.036
0.110
0.169
0.118


In [32]:
np.mean(att_weihgt_1)

0.07142859

In [33]:
Paths_name_AA

['APA', 'APCPA', 'APTPA']

In [34]:
Paths_name_AP

['APCP', 'APTP', 'PA', 'PAPA']

In [35]:
Paths_name_AT

['APCPT', 'TPA', 'TPAPA', 'TPTPA']

In [36]:
Paths_name_AC

['APTPC', 'CPA', 'CPAPA']